In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import matplotlib.pyplot as plt

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Read Data
data_path = "/content/drive/My Drive/Senior Design/Pathology_Project/Student_Version.xlsx"
df = pd.read_excel(data_path)

# Filter out missing samples
df = df.dropna()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Extract features and target
X = []
y = []

for index, sample in df["sample"].items():
    try:
        data = pd.read_csv(f"/content/drive/My Drive/Senior Design/Pathology_Project/713_1217Spectra_csv_Tranformed/sample{sample}.csv",
                           header=None, names=["Column1", "Column2"])
        X.append(data["Column2"])
        y.append(df["Survival"][index])
    except Exception as e:
        pass

In [ ]:
# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

In [ ]:
# Normalize input data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)


In [ ]:
# Neural Network Model
tf.random.set_seed(50)

model = tf.keras.Sequential([
    BatchNormalization(input_shape=(X_train.shape[1],)),
    Dense(100, activation='relu'),
    Dropout(0.2),  # Dropout regularization
    Dense(50, activation='relu'),
    Dropout(0.1),  # Dropout regularization
    Dense(1)
])

model.compile(loss=tf.keras.losses.mae,
              optimizer=tf.keras.optimizers.Adam(),  # Using Adam optimizer for better convergence
              metrics=["mae"])


In [ ]:
# Learning rate scheduler
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-6)


In [ ]:
# Early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping, lr_scheduler])


Epoch 1/100
9/9 [==============================] - 1s 35ms/step - loss: 136.1228 - mae: 136.1228 - val_loss: 132.0211 - val_mae: 132.0211 - lr: 0.0010
Epoch 2/100
9/9 [==============================] - 0s 14ms/step - loss: 113.6963 - mae: 113.6963 - val_loss: 108.3986 - val_mae: 108.3986 - lr: 0.0010
Epoch 3/100
9/9 [==============================] - 0s 11ms/step - loss: 95.7822 - mae: 95.7822 - val_loss: 94.0415 - val_mae: 94.0415 - lr: 0.0010
Epoch 4/100
9/9 [==============================] - 0s 11ms/step - loss: 90.1059 - mae: 90.1059 - val_loss: 92.8525 - val_mae: 92.8525 - lr: 0.0010
Epoch 5/100
9/9 [==============================] - 0s 11ms/step - loss: 89.5302 - mae: 89.5302 - val_loss: 94.4976 - val_mae: 94.4976 - lr: 0.0010
Epoch 6/100
9/9 [==============================] - 0s 12ms/step - loss: 83.9921 - mae: 83.9921 - val_loss: 94.2591 - val_mae: 94.2591 - lr: 0.0010
Epoch 7/100
9/9 [==============================] - 0s 13ms/step - loss: 86.0342 - mae: 86.0342 - val_loss: 93.

In [ ]:
# Plot training history
pd.DataFrame(history.history).plot()
plt.ylabel("Loss Function")
plt.xlabel("Epochs")

In [ ]:
# Evaluate model on test set
mae = model.evaluate(X_test, y_test)[1]
print("Mean Absolute Error:", mae)

In [ ]:
# # Save the improved model
# model.save("/content/drive/My Drive/Bishal/improve_accuracy2.h5")